# COURSE: СЕНЗОРСКИ СИСТЕМИ 
# LAB4
<br/>

N - број на јазли 

L - должина на областа каде е дистрибуирана мрежата 

R - радио опсег 

r - шум  

f - фракција (процент) на anchor јазли 

In [21]:
# IMPORTS
import numpy as np
import math
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import random
from math import cos, sin, pi
import matplotlib.cm as cm
import matplotlib.pyplot as plt
%matplotlib inline

In [52]:
def trilaterate3D(distances):
    p1=np.array(distances[0][:3])
    p2=np.array(distances[1][:3])
    p3=np.array(distances[2][:3])       
    p4=np.array(distances[3][:3])
    
    r1=distances[0][-1]
    r2=distances[1][-1]
    r3=distances[2][-1]
    r4=distances[3][-1]
    
    e_x=(p2-p1)/np.linalg.norm(p2-p1)
    i=np.dot(e_x,(p3-p1))
    e_y=(p3-p1-(i*e_x))/(np.linalg.norm(p3-p1-(i*e_x)))
    e_z=np.cross(e_x,e_y)
    d=np.linalg.norm(p2-p1)
    j=np.dot(e_y,(p3-p1))
    
    x=((r1**2)-(r2**2)+(d**2))/(2*d)
    y=(((r1**2)-(r3**2)+(i**2)+(j**2))/(2*j))-((i/j)*(x))
    
    z1=np.sqrt(np.absolute(r1**2-x**2-y**2))
    z2=np.sqrt(np.absolute(r1**2-x**2-y**2))*(-1)
    
    ans1=p1+(x*e_x)+(y*e_y)+(z1*e_z)
    ans2=p1+(x*e_x)+(y*e_y)+(z2*e_z)
    
    dist1=np.linalg.norm(p4-ans1)
    dist2=np.linalg.norm(p4-ans2)
    
    if np.abs(r4-dist1)<np.abs(r4-dist2):
        return ans1
    else: 
        return ans2

In [85]:
class Node3D:
    """
        
    """
    def __init__(self):
        self.id = None
        self.x_coord = None
        self.y_coord = None
        self.z_coord = None
        self.isAnchor = 0
        self.unreliability = math.inf
        self.x_estimated = None
        self.y_estimated = None
        self.z_estimated = None
        self.neighbours = []
        self.distances = []
        self.estimated = False

    def euclideanDistance(self, node):
        x1 = self.x_coord
        y1 = self.y_coord
        z1 = self.z_coord
        x2 = node.x_coord
        y2 = node.y_coord
        z2 = node.z_coord
        
        result = math.sqrt(math.pow(x2 - x1, 2) + math.pow(y2 - y1, 2) + math.pow(z2 - z1, 2))
        return result
    
    def minUnreliability(self, node):
        if self.unreliability <= node.unreliability:
            return -1
        else:
            return 0

    def getAnchorNeighbours(self):
        anchors = []
        for neighbour in self.neighbours:
            if neighbour.isAnchor > 0:
                anchors.append(neighbour)
        return anchors
    
    def getErrorOfEstimation(self):
        x_diff = math.pow(self.x_estimated - self.x_coord, 2)
        y_diff = math.pow(self.y_estimated - self.y_coord, 2)
        z_diff = math.pow(self.z_estimated - self.z_coord, 2)
        error = math.sqrt( x_diff + y_diff + z_diff)
        return error

    def estimateLocation(self, cycles, heuristic=1, r=10):
        if self.estimated == True:
            return False
        if heuristic == 1:
            #print("Heuristic 1")
            return self.estimateLocation1(cycles, r)
        elif heuristic == 2:
            #print("Heuristic 2")
            return self.estimateLocation2(cycles, r)
        else:
            pass
            
    def __str__(self):
        return "id = " + str(self.id) + "\n" + \
               "\tx = " + str(self.x_coord) + ", y = " + str(self.y_coord) + "\n" + \
               "\tisAnchor = " + str(self.isAnchor) + "\n" + \
               "\tunreliability = " + str(self.unreliability) + "\n" + \
               "\tx_estimated = " + str(self.x_estimated) + ", y = " + str(self.y_estimated) + "\n" + \
               "\tNeighbours: " + str([n.id for n in self.neighbours]) + "\n"

    
    def estimateLocation1(self, cycles, r):
        self.neighbours.sort(key=lambda neighbour: self.euclideanDistance(neighbour))
        anchors = self.getAnchorNeighbours()
        if len(anchors) < 4:
            return False
        else:
            neighbour1 = anchors[0]
            neighbour2 = anchors[1]
            neighbour3 = anchors[2]
            neighbour4 = anchors[3]

            rand = np.random.randint(0, 10)  # to simulate the incorrect distance measured
            noise = None
            if (rand < 5):
                noise = 1 - r / 100.0
            else:
                noise = 1 + r / 100.0

            r1 = self.euclideanDistance(neighbour1) * noise
            r2 = self.euclideanDistance(neighbour2) * noise
            r3 = self.euclideanDistance(neighbour3) * noise
            r4 = self.euclideanDistance(neighbour4) * noise

            pt_a = [neighbour1.x_coord, neighbour1.y_coord, neighbour1.z_coord, r1]
            pt_b = [neighbour2.x_coord, neighbour2.y_coord, neighbour2.z_coord, r2]
            pt_c = [neighbour3.x_coord, neighbour3.y_coord, neighbour3.z_coord, r3]
            pt_d = [neighbour4.x_coord, neighbour4.y_coord, neighbour4.z_coord, r4]
            
            distances = [pt_a, pt_b, pt_c, pt_d]
            
            result = trilaterate3D(distances)

            self.estimated = True
            
            self.x_estimated = result[0]
            self.y_estimated = result[1]
            self.z_estimated = result[2]
            
            self.isAnchor = cycles + 1
            self.unreliability = neighbour1.unreliability + neighbour2.unreliability + neighbour3.unreliability + 1
            
            return True
    
    def estimateLocation2(self, cycles, r):
        self.neighbours.sort(key=lambda neighbour: self.minUnreliability(neighbour))
        anchors = self.getAnchorNeighbours()
        if len(anchors) < 4:
            return False
        else:
            neighbour1 = anchors[0]
            neighbour2 = anchors[1]
            neighbour3 = anchors[2]
            neighbour4 = anchors[3]

            rand = np.random.randint(0, 10)  # to simulate the incorrect distance measured
            noise = None
            if (rand < 5):
                noise = 1 - r / 100.0
            else:
                noise = 1 + r / 100.0

            r1 = self.euclideanDistance(neighbour1) * noise
            r2 = self.euclideanDistance(neighbour2) * noise
            r3 = self.euclideanDistance(neighbour3) * noise
            r4 = self.euclideanDistance(neighbour4) * noise
            
            pt_a = [neighbour1.x_coord, neighbour1.y_coord, neighbour1.z_coord, r1]
            pt_b = [neighbour2.x_coord, neighbour2.y_coord, neighbour2.z_coord, r2]
            pt_c = [neighbour3.x_coord, neighbour3.y_coord, neighbour3.z_coord, r3]
            pt_d = [neighbour4.x_coord, neighbour4.y_coord, neighbour4.z_coord, r4]

            distances = [pt_a, pt_b, pt_c, pt_d]
            
            result = trilaterate3D(distances)

            self.estimated = True
            self.x_estimated = result[0]
            self.y_estimated = result[1]
            self.z_estimated = result[2]
            
            self.isAnchor = cycles + 1
            self.unreliability = neighbour1.unreliability + neighbour2.unreliability + neighbour3.unreliability + 1
            
            return True
    
    
def generateNodes(N, L, R, r, f):
    nodes = dict()
    coordinates = [(float(x) * L, float(y) * L, float(z) * L) for x, y, z in np.random.rand(N, 3)]
    # print(coordinates)
    anchorNodesNumber = int(f / 100 * N)
    for id in range(N):
        node = Node3D()
        node.id = id
        x = coordinates[id][0]
        y = coordinates[id][1]
        z = coordinates[id][2]
        
        node.x_coord = x
        node.y_coord = y
        node.z_coord = z
        
        if id < anchorNodesNumber:
            node.isAnchor = 1
            node.unreliability = 0
            node.x_estimated = node.x_coord
            node.y_estimated = node.y_coord
            node.z_estimated = node.z_coord
            
            node.estimated = True
            
        nodes[id] = node
    return nodes


def plot(nodes, N, L, f):
    anchorNodesNumber = int(f / 100 * N)
    x_anchorNodes = [nodes[i].x_coord for i in range(anchorNodesNumber)]
    y_anchorNodes = [nodes[i].y_coord for i in range(anchorNodesNumber)]
    z_anchorNodes = [nodes[i].z_coord for i in range(anchorNodesNumber)]

    x_other = [nodes[i].x_coord for i in range(anchorNodesNumber + 1, len(nodes))]
    y_other = [nodes[i].y_coord for i in range(anchorNodesNumber + 1, len(nodes))]
    z_other = [nodes[i].z_coord for i in range(anchorNodesNumber + 1, len(nodes))]
    
    anchorNodes = go.Scatter3d(
        x = x_anchorNodes,
        y = y_anchorNodes,
        z = z_anchorNodes,
        mode = "markers",
        marker = dict(
            color = "red", 
            size = 8
        ),
        name = "Anchor Nodes"
    )
    otherNodes = go.Scatter3d(
        x = x_other,
        y = y_other,
        z = z_other,
        mode = "markers",
        marker = dict(
            color = "blue", 
            size = 5
        ),
        name = "Other Nodes"
    )
    
    data = [anchorNodes, otherNodes]

    iplot(data)


def findNeighbours(nodes, R):
    for id in range(len(nodes.keys())):
        neighbours = []
        distances = []
        for id2 in range(len(nodes.keys())):
            if (id == id2):
                continue
            distance = nodes[id].euclideanDistance(nodes[id2])
            if (distance < R):
                neighbours.append(nodes[id2])
                distances.append(distance)
        nodes[id].neighbours = neighbours
        nodes[id].distances = distances
    return nodes



"""
    generateNodes(N, L, R, r, f)
"""
nodes = generateNodes(100, 100, 0, 0, 10)
plot(nodes, 100, 100, 10)


"""
    findNeighbours(nodes, R)
"""
nodes = findNeighbours(nodes, 35)

In [86]:
def estimateLocations(nodes, heuristic, r = 10):
    global cycles 
    cycles = 0
    while (True):
        cycles = cycles + 1
        estimatedSmth = False
        for idx in range(len(nodes)):
            # print(nodes[idx].estimateLocation())
            est = nodes[idx].estimateLocation(cycles, heuristic, r)
            if est:
                # print(nodes[idx])
                estimatedSmth = True
                nodes[idx].estimated = True
        if not estimatedSmth:
            break
    return nodes

nodes = estimateLocations(nodes, 1)
            
black_nodes = []
anchor_nodes = []
estimated_nodes = []


for idx in range(len(nodes)):
    if nodes[idx].isAnchor == 1:
        anchor_nodes.append(nodes[idx])
    elif nodes[idx].estimated:
        estimated_nodes.append(nodes[idx])
    else:
        black_nodes.append(nodes[idx])

print("Total of Black Nodes:", len(black_nodes))
print("Total of Anchor Nodes:", len(anchor_nodes))
print("Total of Estimated Nodes:", len(estimated_nodes))
print("Cycles needed to estimate as much as possible nodes: ", cycles)

Total of Black Nodes: 17
Total of Anchor Nodes: 10
Total of Estimated Nodes: 73
Cycles needed to estimate as much as possible nodes:  6


In [87]:
"""
    This is a function that plots the Anchor Nodes (RED), Nodes that could have been estimated (YELLOW)
    and nodes that couldn't have been estimated (BLACK)

"""

def plot2(black_nodes, anchor_nodes, estimated_nodes, cycles):
    x_black_nodes = [black_nodes[i].x_coord for i in range(len(black_nodes))]
    y_black_nodes = [black_nodes[i].y_coord for i in range(len(black_nodes))]
    z_black_nodes = [black_nodes[i].z_coord for i in range(len(black_nodes))]

    x_anchor_nodes = [anchor_nodes[i].x_coord for i in range(len(anchor_nodes))]
    y_anchor_nodes = [anchor_nodes[i].y_coord for i in range(len(anchor_nodes))]
    z_anchor_nodes = [anchor_nodes[i].z_coord for i in range(len(anchor_nodes))]
    
    x_estimated_nodes = [estimated_nodes[i].x_coord for i in range(len(estimated_nodes))]
    y_estimated_nodes = [estimated_nodes[i].y_coord for i in range(len(estimated_nodes))]
    z_estimated_nodes = [estimated_nodes[i].z_coord for i in range(len(estimated_nodes))]
    
    name_black = "Black Nodes (" + str(len(black_nodes)) + ")"
    black_nodes = go.Scatter3d(
        x = x_black_nodes,
        y = y_black_nodes,
        z = z_black_nodes,
        mode = "markers",
        marker = dict(
            color = "black", 
            size = 5),
        name = name_black
    )
    
    name_anchor = "Anchor Nodes (" + str(len(anchor_nodes)) + ")"
    anchor_nodes = go.Scatter3d(
        x = x_anchor_nodes,
        y = y_anchor_nodes,
        z = z_anchor_nodes,
        mode = "markers",
        marker = dict(
            color = "red", 
            size = 8),
        name = name_anchor
    )
    
    name_estimated = "Estimated Nodes (" + str(len(estimated_nodes)) + ")"
    estimated_nodes = go.Scatter3d(
        x = x_estimated_nodes,
        y = y_estimated_nodes,
        z = z_estimated_nodes,
        mode = "markers",
        marker = dict(
            color = "yellow", 
            size = 5),
        name = name_estimated
    )
    
    data = [black_nodes, anchor_nodes, estimated_nodes]

    iplot(data)
    

    
"""
    This is a function that plots the correct location of a node (red) and the estimated location of the
    node by trilateration (blue)
"""
def plot3(original_node, estimated_node):
    x_original_node = [original_node[i][0] for i in range(len(original_node))]
    y_original_node = [original_node[i][1] for i in range(len(original_node))]
    z_original_node = [original_node[i][2] for i in range(len(original_node))]

    x_estimated_node = [estimated_node[i][0] for i in range(len(estimated_node))]
    y_estimated_node = [estimated_node[i][1] for i in range(len(estimated_node))]
    z_estimated_node = [estimated_node[i][2] for i in range(len(estimated_node))]
    
    original_node_name = "Correct Location"
    original_nodes = go.Scatter3d(
        x = x_original_node,
        y = y_original_node,
        z = z_original_node,
        mode = "markers",
        marker = dict(
            color = "red", 
            size = 5),
        name = original_node_name
    )
    
    estimated_node_name = "Estimated Node"
    estimated_nodes = go.Scatter3d(
        x = x_estimated_node,
        y = y_estimated_node,
        z = z_estimated_node,
        mode = "markers",
        marker = dict(
            color = "blue", 
            size = 5),
        name = estimated_node_name
    )
    
    data = [original_nodes, estimated_nodes]

    iplot(data)
    

original_node = []
estimated_node = []


for node in estimated_nodes:
    node_original = node.x_coord, node.y_coord, node.z_coord
    node_estimated = node.x_estimated, node.y_estimated, node.z_estimated
    
    original_node.append(node_original)
    estimated_node.append(node_estimated)

x_original_node = [original_node[i][0] for i in range(len(original_node))]
y_original_node = [original_node[i][1] for i in range(len(original_node))]
z_original_node = [original_node[i][2] for i in range(len(original_node))]

x_estimated_node = [estimated_node[i][0] for i in range(len(estimated_node))]
y_estimated_node = [estimated_node[i][1] for i in range(len(estimated_node))]
z_estimated_node = [estimated_node[i][2] for i in range(len(estimated_node))]

In [89]:
plot2(black_nodes, anchor_nodes, estimated_nodes, cycles)

In [90]:
plot3(original_node, estimated_node)

In [95]:
data = []

original_node_name = "Correct Location"
original_nodes = go.Scatter3d(
        x = x_original_node,
        y = y_original_node,
        z = z_original_node,
        mode = "markers",
        marker = dict(
            color = "red", 
            size = 5),
        name = original_node_name
    )

estimated_node_name = "Estimated Nodes"
estimated_nodes1 = go.Scatter3d(
        x = x_estimated_node,
        y = y_estimated_node,
        z = z_estimated_node,
        mode = "markers",
        marker = dict(
            color = "blue", 
            size = 5),
        name = estimated_node_name
    )   

x_black_nodes = [black_nodes[i].x_coord for i in range(len(black_nodes))]
y_black_nodes = [black_nodes[i].y_coord for i in range(len(black_nodes))]
z_black_nodes = [black_nodes[i].z_coord for i in range(len(black_nodes))]

name_black = "Black Nodes (" + str(len(black_nodes)) + ")"
black_nodes = go.Scatter3d(
        x = x_black_nodes,
        y = y_black_nodes,
        z = z_black_nodes,
        mode = "markers",
        marker = dict(
            color = "black", 
            size = 5),
        name = name_black
    )   

data.append(original_nodes)
data.append(estimated_nodes1)
data.append(black_nodes)

for node in estimated_nodes:
    trace = go.Scatter3d(
        x = [node.x_coord, node.x_estimated],
        y = [node.y_coord, node.y_estimated],
        z = [node.z_coord, node.z_estimated],
        mode = 'lines',
        name = 'Node ' + str(node.id),
        marker = dict(
            size = 10,
            color = 'black',
            line = dict(
                width = 1,
                color = 'black'
            )
        )
    )
    data.append(trace)

In [96]:
iplot(data)

In [97]:
def calculateError(nodes, N, f):
    estimated = 0
    error = 0
    for idx in range(len(nodes)):
        if nodes[idx].isAnchor > 1:
            estimated = estimated + 1
            error = error + nodes[idx].getErrorOfEstimation()
    return error/(N-f)

# PLOT 1

## Фракција на anchor nodes vs Error

In [98]:
N = 100
L = 100
R = 30
r = 10
# f = {10,20,30,40,50}
heuristic = 1
errorsH1 = []

nodes = None
for f in range(10, 51, 10):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/20)

print(errorsH1)

[1.2002957913127241, 3.8467576364772924, 4.283199226902201, 4.399272261378266, 4.602961419070341]


In [99]:
heuristic = 2
errorsH2 = []

nodes = None
for f in range(10, 51, 10):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/20)

print(errorsH2)

[1.0381026961785664, 3.91745200098516, 4.791994442875614, 5.622392230699205, 5.582784540949056]


In [100]:
f = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = f,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = f,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)

# PLOT 2

## Радио опсег vs Error

In [101]:
N = 100
L = 100
f = 25
r = 10
# R = {20, 30, 40, 50}

heuristic = 1
errorsH1 = []

nodes = None
for R in range(20, 51, 10):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/20)

print(errorsH1)

[0.05001305904945939, 3.6305043242428865, 6.732077643335342, 6.728209737784343]


In [102]:
heuristic = 2
errorsH2 = []

nodes = None
for R in range(20, 51, 10):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/20)

print(errorsH2)

[0.05188861850342319, 4.696276089353523, 8.393086614479802, 10.588179903049946]


In [103]:
R = [10, 20, 30, 40, 50]

trace0 = go.Scatter(
    x = R,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = R,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)

# PLOT 3

## Шум vs Error

In [104]:
N = 100
L = 100
f = 25
R = 35
# r = {5,10,15,20,25}

heuristic = 1
errorsH1 = []

nodes = None
for r in range(5, 26, 5):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, r)
        error = error + calculateError(nodes, N, f)
    errorsH1.append(error/20)

print(errorsH1)

[3.3409312430402514, 5.94993649701099, 8.097632033503347, 10.597336788368056, 12.31137501082885]


In [105]:
heuristic = 2
errorsH2 = []

nodes = None
for r in range(5, 26, 5):
    error = 0
    for i in range(0, 20):
        nodes = generateNodes(N, L, R, r, f)
        nodes = findNeighbours(nodes, R)
        nodes = estimateLocations(nodes, heuristic, r)
        error = error + calculateError(nodes, N, f)
    errorsH2.append(error/20)

print(errorsH2)

[4.107603613570088, 7.552460650952637, 11.195685150230394, 12.5642470633764, 15.957918241035662]


In [106]:
r = [5, 10, 15, 20, 25]

trace0 = go.Scatter(
    x = r,
    y = errorsH1,
    mode = 'lines',
    name = 'Heuristic1'
)

trace1 = go.Scatter(
    x = r,
    y = errorsH2,
    mode = 'lines',
    name = 'Heuristic2'
)

data = [trace0, trace1]

iplot(data)